# Quick setup to get started with Airlines DB

#### Install MongoDB
Depending on current OS

#### Install PyMongo
`pip install pymongo`

#### Start MongoDB
`sudo systemctl start mongod`


## About the DB
This collection contains data of USA airports from 2003 to 2016. 


Each document mainly contains info about flights and delays, divided by Airport, Month and Year.

To find out more visit the [CORGIS Dataset Project](https://think.cs.vt.edu/corgis/json/airlines/)




# Connect and select DB

Using PyMongo we can easily connect to MongoDB, and then select the DB and Collection from the list.

In [4]:
from pymongo import MongoClient

try:
    client = MongoClient('localhost', 27017)
    print("Connected successfully!")
except:
    print("Could not connect to MongoDB")    

print(client.list_database_names())

db = client.airlines

try:
    db.create_collection('delays')
except:
    print('Already existing')
print(db.list_collection_names())

delays = db.delays

Connected successfully!
['admin', 'config', 'local']
['delays']


# Load data into DB


#### Insert one object

In [2]:
obj = {"Uno":1}
delays.insert_one(obj)

### Drop Collection. 

In [3]:
try:
    delays.drop()
    print('Deleted')
except:
    print('Error deleting Collection')

Deleted


#### Insert data from file

In [5]:
import json

with open('airlines.json') as f:
    data = json.load(f)

delays.insert_many(data)

#### To find all the records

Iterate and print the first 3 documents

In [6]:
from pprint import pprint
i = 0
for doc in delays.find():
    while i < 3:
        pprint(doc)
        i+=1

{'Airport': {'Code': 'ATL',
             'Name': 'Atlanta, GA: Hartsfield-Jackson Atlanta International'},
 'Statistics': {'# of Delays': {'Carrier': 1009,
                                'Late Aircraft': 1275,
                                'National Aviation System': 3217,
                                'Security': 17,
                                'Weather': 328},
                'Carriers': {'Names': 'American Airlines Inc.,JetBlue '
                                      'Airways,Continental Air Lines '
                                      'Inc.,Delta Air Lines Inc.,Atlantic '
                                      'Southeast Airlines,AirTran Airways '
                                      'Corporation,America West Airlines '
                                      'Inc.,Northwest Airlines Inc.,ExpressJet '
                                      'Airlines Inc.,United Air Lines Inc.,US '
                                      'Airways Inc.',
                             'Total': 11}

#### Find the first record matching the query

In [7]:
pprint(delays.find_one({"Airport.Code": "ATL",
                              "Time.Month" : 6}))

{'Airport': {'Code': 'ATL',
             'Name': 'Atlanta, GA: Hartsfield-Jackson Atlanta International'},
 'Statistics': {'# of Delays': {'Carrier': 1009,
                                'Late Aircraft': 1275,
                                'National Aviation System': 3217,
                                'Security': 17,
                                'Weather': 328},
                'Carriers': {'Names': 'American Airlines Inc.,JetBlue '
                                      'Airways,Continental Air Lines '
                                      'Inc.,Delta Air Lines Inc.,Atlantic '
                                      'Southeast Airlines,AirTran Airways '
                                      'Corporation,America West Airlines '
                                      'Inc.,Northwest Airlines Inc.,ExpressJet '
                                      'Airlines Inc.,United Air Lines Inc.,US '
                                      'Airways Inc.',
                             'Total': 11}

# Update documents

#### Update one, according to query

In [8]:
query = {'Airport.Code' : 'ATL',
        'Time.Year' : 2005}

x = delays.update_one(query, {'$set' : {'Airport.Name' : 'Atlanta Airport - 2005'}})

print(x.modified_count, "documents updated.") 

1 documents updated.


#### Update all documents matching the query

In [9]:
query = {'Airport.Code' : 'ATL'}

x = delays.update_many(query, {'$set' : {'Airport.Name' : 'Atlanta International'}})

print(x.modified_count, "documents updated.") 

152 documents updated.


# How to query the Collection

#### 1. Search for the Airports with the highest number of Total Delays, during January 2010. Descending order, limit to 5 results.

In [10]:
docs = delays.find({'$and': [{'Time.Year' : 2010}, 
                             {'Time.Month' : 1}]},
                   {'_id':False, 
                    'Statistics.# of Delays':False,
                    'Statistics.Carriers':False,
                    'Statistics.Minutes Delayed':False,
                    'Time':False
                   }).sort('Statistics.Flights.Total',-1).limit(5)
pprint(list(docs))

[{'Airport': {'Code': 'ATL', 'Name': 'Atlanta International'},
  'Statistics': {'Flights': {'Cancelled': 1148,
                             'Delayed': 5908,
                             'Diverted': 74,
                             'On Time': 26599,
                             'Total': 33729}}},
 {'Airport': {'Code': 'ORD',
              'Name': "Chicago, IL: Chicago O'Hare International"},
  'Statistics': {'Flights': {'Cancelled': 927,
                             'Delayed': 5164,
                             'Diverted': 20,
                             'On Time': 19349,
                             'Total': 25460}}},
 {'Airport': {'Code': 'DFW',
              'Name': 'Dallas/Fort Worth, TX: Dallas/Fort Worth International'},
  'Statistics': {'Flights': {'Cancelled': 590,
                             'Delayed': 3550,
                             'Diverted': 97,
                             'On Time': 18319,
                             'Total': 22556}}},
 {'Airport': {'Code': 'DEN', '

#### 2. Report how many flights were cancelled at Los Angeles Airport, every year between 2003 - 2016

In [11]:
docs = delays.aggregate([{'$match': {'Airport.Code' : 'LAX'}},
                         {'$group': {'_id' : '$Time.Year',
                                     'Cancelled Flights' : {'$sum' : '$Statistics.Flights.Cancelled'}}},
                         {'$sort': {'_id' : 1}} ])

for doc in docs:
    pprint(doc)

{'Cancelled Flights': 1654, '_id': 2003}
{'Cancelled Flights': 2726, '_id': 2004}
{'Cancelled Flights': 2592, '_id': 2005}
{'Cancelled Flights': 3248, '_id': 2006}
{'Cancelled Flights': 3417, '_id': 2007}
{'Cancelled Flights': 3077, '_id': 2008}
{'Cancelled Flights': 2000, '_id': 2009}
{'Cancelled Flights': 2292, '_id': 2010}
{'Cancelled Flights': 2764, '_id': 2011}
{'Cancelled Flights': 2493, '_id': 2012}
{'Cancelled Flights': 2541, '_id': 2013}
{'Cancelled Flights': 2726, '_id': 2014}
{'Cancelled Flights': 2337, '_id': 2015}
{'Cancelled Flights': 297, '_id': 2016}


#### 3. Find the 5 airports with the highest number of flights delayed due to weather conditions

In [12]:
docs = delays.aggregate([
    {'$group': {'_id' : '$Airport.Code',
                'Name' : {'$first' : '$Airport.Name'},
                'Weather Delayed': {'$sum' : '$Statistics.# of Delays.Weather'}}},
    {'$sort': {'Weather Delayed' : -1}},
    {'$limit': 5}
])

for doc in docs:
    pprint(doc)

{'Name': 'Atlanta International', 'Weather Delayed': 40113, '_id': 'ATL'}
{'Name': 'Dallas/Fort Worth, TX: Dallas/Fort Worth International',
 'Weather Delayed': 30476,
 '_id': 'DFW'}
{'Name': "Chicago, IL: Chicago O'Hare International",
 'Weather Delayed': 24358,
 '_id': 'ORD'}
{'Name': 'New York, NY: LaGuardia', 'Weather Delayed': 16350, '_id': 'LGA'}
{'Name': 'Denver, CO: Denver International',
 'Weather Delayed': 15556,
 '_id': 'DEN'}
